In [1]:
import requests
from bs4 import BeautifulSoup
from riotwatcher import LolWatcher, ApiError
import json
from collections import defaultdict
from lxml import etree
from constants import n2id, LOL_API_KEY, logger

In [120]:
def get_champions_played(region, summoner_name, champions, last_n=20):
    try:
        watcher = LolWatcher(LOL_API_KEY)
        summoner = watcher.summoner.by_name(region, summoner_name)
        ranked_data = watcher.league.by_summoner(region, summoner["id"])
        match_history = watcher.match.matchlist_by_puuid(region, summoner["puuid"])
        last_20_matches = match_history[:last_n]
        for match in last_20_matches:
            match_data = watcher.match.by_id(match_id=match, region=region)
            for player in dict(match_data)["info"]["participants"]:
                if player["puuid"] == summoner["puuid"]:
                    champ_name = player["championName"]
                    win = 1 if player["win"] else 0
                    stat = champions[champ_name]
                    stat = (stat[0] + 1, stat[1] + win)
                    champions[champ_name] = stat
        return champions
    except ApiError as err:
        print(f"Error while retrieving summoner information: {err}")
        return None, []


def get_rank_by_queue(region, summoner_name, queue):
    try:
        watcher = LolWatcher(LOL_API_KEY)
        summoner = watcher.summoner.by_name(region, summoner_name)
        ranked_data = watcher.league.by_summoner(region, summoner["id"])
        for queue_data in ranked_data:
            if queue_data["queueType"] == queue:
                rank = (
                    queue_data["tier"]
                    + " "
                    + queue_data["rank"]
                    + " "
                    + str(queue_data["leaguePoints"])
                )
                break
        else:
            rank = "Unranked"
        return rank
    except ApiError as err:
        print(f"Error while retrieving summoner information: {err}")
        return "Unranked"


def get_ranked_stats(summoner_name, region="euw1", last_n=20):
    champions = defaultdict(lambda: (0, 0))
    soloq_rank = get_rank_by_queue(region, summoner_name, "RANKED_SOLO_5x5")
    flex_rank = get_rank_by_queue(region, summoner_name, "RANKED_FLEX_SR")
    champions = get_champions_played(region, summoner_name, champions, last_n=last_n)
    return soloq_rank, flex_rank, champions

In [129]:
def calculate_win_rate(value):
    total_games, wins = value
    return round(wins / total_games * 100, 2) if total_games != 0 else 0


def get_player_data_by_lolpros(url, last_n=20):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    account_elements = soup.find_all(class_="account")
    account_ids = [element.text.strip() for element in account_elements]
    player_data = {"accounts": [], "history": []}
    player_champs = defaultdict(lambda: (0, 0))
    for account_id in account_ids:
        opgg = f"https://www.op.gg/summoners/euw/{account_id}"
        soloq_rank, flex_rank, champions = get_ranked_stats(
            summoner_name=account_id, last_n=last_n
        )
        player_data["accounts"].append(
            {
                "account_name": account_id,
                "opgg": opgg,
                "soloq": soloq_rank,
                "flexq": flex_rank,
            }
        )

        for champ, value in champions.items():
            player_champs[champ] = (value[0], calculate_win_rate(value))

    player_champs = dict(
        sorted(player_champs.items(), key=lambda item: item[1], reverse=True)
    )
    for champ, stats in player_champs.items():
        player_data["history"].append(
            {
                "champion": champ,
                "played": stats[0],
                "win_rate": stats[1],
                "icon": f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/{n2id.get(champ,-1)}.png",
            }
        )
    return player_data


url = "https://lolpros.gg/player/goksi"
player_data = get_player_data_by_lolpros(url, last_n=20)
player_data

{'accounts': [{'account_name': 'Goksiáš',
   'opgg': 'https://www.op.gg/summoners/euw/Goksiáš',
   'soloq': 'MASTER I 356',
   'flexq': 'Unranked'},
  {'account_name': 'Jiàn de lìliàng',
   'opgg': 'https://www.op.gg/summoners/euw/Jiàn de lìliàng',
   'soloq': 'PLATINUM I 76',
   'flexq': 'SILVER III 60'}],
 'history': [{'champion': 'Kindred',
   'played': 13,
   'win_rate': 76.92,
   'icon': 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/203.png'},
  {'champion': 'Rumble',
   'played': 3,
   'win_rate': 66.67,
   'icon': 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/68.png'},
  {'champion': 'Diana',
   'played': 3,
   'win_rate': 33.33,
   'icon': 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/131.png'},
  {'champion': 'Vi',
   'played': 2,
   'win_rate': 50.0,
   'icon': 'https://raw.communitydragon.org/latest/plugins/rc

In [116]:
def get_team_data_from_lol_wiki(team):
    query = f"""https://lol.fandom.com/wiki/Special:RunQuery/MatchHistoryGame?MHG%5Bpreload%5D=Tournament&MHG%5Btournament%5D=Hitpoint+Masters%2F2023+Season%2FSpring+Season&MHG%5Bteam%5D={team}&MHG%5Bteam1%5D=&MHG%5Bteam2%5D=&MHG%5Bban%5D=&MHG%5Brecord%5D=&MHG%5Bascending%5D%5Bis_checkbox%5D=true&MHG%5Blimit%5D=&MHG%5Boffset%5D=&MHG%5Bregion%5D=&MHG%5Byear%5D=&MHG%5Bstartdate%5D=&MHG%5Benddate%5D=&MHG%5Bwhere%5D=&MHG%5Btextonly%5D%5Bis_checkbox%5D=true&_run=&pfRunQueryFormName=MatchHistoryGame"""
    response = requests.get(query)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find_all(class_="wide-content-scroll")[0].find("table")
    table_data = []
    for row in table.find_all("tr"):
        row_data = []
        for cell in row.find_all("td"):
            row_data.append(cell)
        table_data.append(row_data)
    df_data = []
    team_picks = {
        "top": defaultdict(lambda: 0),
        "jungle": defaultdict(lambda: 0),
        "mid": defaultdict(lambda: 0),
        "adc": defaultdict(lambda: 0),
        "supp": defaultdict(lambda: 0),
    }
    bans = defaultdict(lambda: 0)
    bans_against = defaultdict(lambda: 0)
    data_found = 0
    for row_data in table_data:
        if row_data == []:
            continue
        data_found = 1
        row = {
            "date": row_data[0].text,
            "blue": row_data[2].find("a")["title"],
            "red": row_data[3].find("a")["title"],
            "winner": row_data[4].find("a")["title"],
            "bans_red": [c["title"] for c in row_data[5].find_all("span")],
            "bans_blue": [c["title"] for c in row_data[6].find_all("span")],
            "picks_red": [c["title"] for c in row_data[7].find_all("span")],
            "picks_blue": [c["title"] for c in row_data[8].find_all("span")],
        }
        side = "blue" if team in str(row["blue"]).lower() else "red"
        oponent = "red" if side == "blue" else "blue"
        roles = list(team_picks.keys())
        for i, champ in enumerate(row[f"picks_{side}"]):
            team_picks[roles[i]][champ] = team_picks[roles[i]][champ] + 1

        for champ in row[f"bans_{side}"]:
            bans[champ] = bans[champ] + 1

        for champ in row[f"bans_{oponent}"]:
            bans_against[champ] = bans_against[champ] + 1
    if not data_found:
        return [], [], []

    for role in roles:
        team_picks[role] = dict(
            sorted(team_picks[role].items(), key=lambda item: item[1], reverse=True)
        )
    bans = dict(sorted(bans.items(), key=lambda item: item[1], reverse=True))
    bans_against = dict(
        sorted(bans_against.items(), key=lambda item: item[1], reverse=True)
    )

    team_picks_processed = {
        "top": [],
        "jungle": [],
        "mid": [],
        "adc": [],
        "supp": [],
    }
    bans_processed = []
    bans_against_processed = []
    for role in roles:
        for c in team_picks[role]:
            name = c
            pick = team_picks[role][name]
            team_picks_processed[role].append(
                {
                    "name": c,
                    "picks": pick,
                    "icon": f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/{n2id[name]}.png",
                }
            )
    for b in bans:
        bans_processed.append(
            {
                "name": b,
                "count": bans[b],
                "icon": f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/{n2id[name]}.png",
            }
        )
    for b in bans_against:
        bans_against_processed.append(
            {
                "name": b,
                "count": bans_against[b],
                "icon": f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/{n2id[name]}.png",
            }
        )
    return team_picks_processed, bans_processed, bans_against_processed

In [117]:
get_team_data_from_lol_wiki("estuba")

([], [], [])